# Q&H promotes in April but Unilock does not promote throughout the year

In [2]:
import pulp
import pandas as pd

In [3]:
# Data
demand = [280, 301, 277, 510, 285, 278, 291, 220, 304, 295, 302, 297]
months = range(12)
production_capacity = 32000
initial_inventory = 150
final_inventory = 150
safety_stock = 100
production_cost = 1010  # $10 (labor) + $1000 (material)
inventory_cost = 100
subcontracting_cost = 1200
selling_price = [2600] * 12  # Default selling price per ton

In [4]:
# Promotion Settings
qh_promotion_month = 3  # Set to None if Q&H does not promote
unilock_promotion_month = None  # Set to None if Unilock does not promote

if qh_promotion_month is not None:
    selling_price[qh_promotion_month] = 2340  # Q&H discount
    if unilock_promotion_month is None or unilock_promotion_month != qh_promotion_month:  # Only Q&H promotes
        demand[qh_promotion_month] = 1.5 * demand[qh_promotion_month] + 0.2 * (demand[qh_promotion_month + 1] + demand[qh_promotion_month + 2])
        demand[qh_promotion_month + 1] *= 0.8
        demand[qh_promotion_month + 2] *= 0.8
    elif unilock_promotion_month == qh_promotion_month:  # Both promote
        demand[qh_promotion_month] = demand[qh_promotion_month] + 0.25 * (demand[qh_promotion_month + 1] + demand[qh_promotion_month + 2])
        demand[qh_promotion_month + 1] *= 0.75
        demand[qh_promotion_month + 2] *= 0.75

if unilock_promotion_month is not None:
    if qh_promotion_month is None or qh_promotion_month != unilock_promotion_month:  # Only Unilock promotes
        demand[unilock_promotion_month] *= 0.5
    elif qh_promotion_month == unilock_promotion_month:  # Both promote (already handled above)
        pass

In [5]:
# Model
model = pulp.LpProblem("Aggregate_Production_Planning", pulp.LpMaximize)

In [6]:
# Decision Variables
P = pulp.LpVariable.dicts("Production", months, lowBound=0, cat='Continuous')
I = pulp.LpVariable.dicts("Inventory", months, lowBound=0, cat='Continuous')
S = pulp.LpVariable.dicts("Subcontracting", months, lowBound=0, cat='Continuous')

In [7]:
# Objective Function (Profit Maximization)
model += pulp.lpSum((selling_price[t] - production_cost) * P[t] + (selling_price[t] - subcontracting_cost) * S[t] - inventory_cost * I[t] for t in months)

In [8]:
# Constraints
# Production Capacity
for t in months:
    model += P[t] <= production_capacity

# Inventory Balance
for t in range(1, 12):
    model += I[t] == I[t-1] + P[t] + S[t] - demand[t]

# Initial Inventory
model += I[0] == initial_inventory + P[0] + S[0] - demand[0]

# Final Inventory
model += I[11] == final_inventory

# Safety Stock Constraint
for t in months:
    model += I[t] >= safety_stock

In [9]:
# Solve
model.solve()

1

In [10]:
# Results
print("Status:", pulp.LpStatus[model.status])
for t in months:
    print(f"Month {t+1}: Production={P[t].varValue}, Inventory={I[t].varValue}, Subcontracting={S[t].varValue}")

print("Total Profit:", pulp.value(model.objective))

Status: Optimal
Month 1: Production=230.0, Inventory=100.0, Subcontracting=0.0
Month 2: Production=301.0, Inventory=100.0, Subcontracting=0.0
Month 3: Production=1154.6, Inventory=977.6, Subcontracting=0.0
Month 4: Production=0.0, Inventory=100.0, Subcontracting=0.0
Month 5: Production=228.0, Inventory=100.0, Subcontracting=0.0
Month 6: Production=222.4, Inventory=100.0, Subcontracting=0.0
Month 7: Production=291.0, Inventory=100.0, Subcontracting=0.0
Month 8: Production=220.0, Inventory=100.0, Subcontracting=0.0
Month 9: Production=304.0, Inventory=100.0, Subcontracting=0.0
Month 10: Production=295.0, Inventory=100.0, Subcontracting=0.0
Month 11: Production=302.0, Inventory=100.0, Subcontracting=0.0
Month 12: Production=347.0, Inventory=150.0, Subcontracting=0.0
Total Profit: 5980290.0
